# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import clustering4ever.scala.clustering.kmeans.{KMeans, KMeansModel}
import clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import clustering4ever.scala.indexes.ExternalIndexes
import clustering4ever.scala.indexes.NmiNormalizationNature._
import clustering4ever.util.ScalaImplicits._
import clustering4ever.scala.clusterizables.RealClusterizable
import clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-09-09 23:24:01--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... failed: Name or service not known.
wget: unable to resolve host address ‘www.clustering4ever.org’
:sh: Scheme missing.
--2018-09-09 23:24:31--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-09-09 23:24:31--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.20’

     0K .                                                     100%  154M=0s

2018-09-09 23:24:31 (154 MB/s) - ‘/tmp/labels.20’ saved [1576/1576]

FINISHED --2018-09-09 23:24:31--
Total wall clock time: 30s
Downloaded: 1 files, 1.5K 

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 2000000
val dim = 100
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new RealClusterizable(id.toLong, new RealVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 2000000
dim: Int = 100
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(15.55, 28.65), ArrayBuffer(14.9, 27.55), ArrayBuffer(14.45, 28.35), ArrayBuffer(14.15, 28.8), ArrayBuffer(13.75, 28.05), ArrayBuffer(13.35, 28.45), ArrayBuffer(13.0, 29.15), ArrayBuffer(13.45, 27.5), ArrayBuffer(13.6, 26.5), ArrayBuffer(12.8, 27.35), ArrayBuffer(12.4, 27.85), ArrayBuffer(12.3, 28.4), ArrayBuffer(12.2, 28.65), ArrayBuffer(13.4, 25.1), ArrayBuffer(12.95, 25.95), ArrayBuffer(12.9, 26.5), ArrayBuffer(11.85, 27.0), ArrayBuffer(11.35, 28.0), ArrayBuffer(11.15, 28.7), ArrayBuffer(11.25, 27.4), ArrayBuffer(10.75, 27.7), ArrayBuffer(10.5, 28.35), ArrayBuffer(...

## Parameters 

In [ ]:
val k = 7
val iterMax = 40
val epsilon = 0.5
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](true)

// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](2)

k: Int = 7
iterMax: Int = 40
epsilon: Double = 0.5
metric1: clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance with root applied
metric2: clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = clustering4ever.math.distances.scalar.Minkowski@416cdee4


## Run and measure the algorithm time

In [ ]:
val t1 = System.nanoTime

val model = KMeans.run[
              Long,
              mutable.ArrayBuffer[Double],
              mutable.ArrayBuffer[Double],
              RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
              Euclidean[mutable.ArrayBuffer[Double]]
              ](
                parData,
                k,
                epsilon,
                iterMax,
                metric1
              )

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 50737379175767
model: clustering4ever.scala.clustering.kmeans.KMeansModel[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double],clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]],clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]]] = clustering4ever.scala.clustering.kmeans.KMeansModel@488f7fcf
t2: Long = 50737674595978
res6: Double = 0.295420211


0.295420211

# Compare algorithm between ParArray and ArraySeq container

In [ ]:
val t1 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime

KMeans.run[Long,
    mutable.ArrayBuffer[Double],
    mutable.ArrayBuffer[Double],
    RealClusterizable[Long, mutable.ArrayBuffer[Double], mutable.ArrayBuffer[Double]],
    Euclidean[mutable.ArrayBuffer[Double]]
    ](seqData, k, epsilon, iterMax, metric1)

val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 50740894588218
t2: Long = 50740936038049
seqData: scala.collection.mutable.Seq[clustering4ever.scala.clusterizables.RealClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(RealClusterizable(0,clustering4ever.scala.vectorizables.RealVector@112154fe,None,None), RealClusterizable(1,clustering4ever.scala.vectorizables.RealVector@557886da,None,None), RealClusterizable(2,clustering4ever.scala.vectorizables.RealVector@6d7753eb,None,None), RealClusterizable(3,clustering4ever.scala.vectorizables.RealVector@78355e89,None,None), RealClusterizable(4,clustering4ever.scala.vectorizables.RealVector@632863d5,None,None), RealClusterizable(5,clustering4ever.scala.vectorizables.RealVector@7277773c,None,None), RealClusterizabl...

0.44028546220128134

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((6,ArrayBuffer(15.55, 28.65)), (3,ArrayBuffer(14.9, 27.55)), (3,ArrayBuffer(14.45, 28.35)), (3,ArrayBuffer(14.15, 28.8)), (3,ArrayBuffer(13.75, 28.05)), (3,ArrayBuffer(13.35, 28.45)), (3,ArrayBuffer(13.0, 29.15)), (3,ArrayBuffer(13.45, 27.5)), (3,ArrayBuffer(13.6, 26.5)), (3,ArrayBuffer(12.8, 27.35)), (3,ArrayBuffer(12.4, 27.85)), (3,ArrayBuffer(12.3, 28.4)), (3,ArrayBuffer(12.2, 28.65)), (3,ArrayBuffer(13.4, 25.1)), (3,ArrayBuffer(12.95, 25.95)), (3,ArrayBuffer(12.9, 26.5)), (3,ArrayBuffer(11.85, 27.0)), (3,ArrayBuffer(11.35, 28.0)), (3,ArrayBuffer(11.15, 28.7)), (3,ArrayBuffer(11.25, 27.4)), (3,ArrayBuffer(10.75, 27.7)), (3,ArrayBuffer(10.5, 28.35)), (3,ArrayBu...

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID.get ).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,34,1000x966,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x966,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID.get).toParArray

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

trueLabels: scala.collection.parallel.mutable.ParArray[Int] = ParArray(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...

0.8538856763962724